### author by yangshichen
### 注意：脚本仅供参考，使用前请仔细阅读

In [4]:
library(ggplot2)
library(future)
library(tidyverse)
library(ggpubr)
library(ggchicklet)
library(ggsci)
library(magrittr)
library(ggh4x)
library(rstatix)
library(ggsignif)
library(ggnewscale)
library(patchwork)
library(gapminder)
library(ggprism)
library(dplyr)
library(ggplotify)
library(readr)
library(arrow)
library(ggbreak)
library(lme4)
library(Matrix)
library(lmtest)
library(pbmcapply)
library(data.table)
library(arrow)
library(Seurat)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

载入程序包：‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract



载入程序包：‘rstatix’


The following object is masked from ‘package:stats’:

    filter



载入程序包：‘arrow’


The following object is masked from ‘package:magrittr’:

    is_in


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp


ggbreak v0.1.5 Learn more 

In [5]:
mytheme <- theme_prism(base_family="",base_fontface="plain") +
  theme(strip.text = element_text(size = 8,angle=10,vjust = 0.5,hjust = 0.5),
        axis.line = element_line(color = "black",size = 0.2),
        axis.ticks = element_line(size = 0.2),
        axis.text.y = element_text(color = "black",size = 6),
        axis.text.x = element_text(color = "black",size = 6, angle = 30,hjust = 1,vjust = 1),
        axis.title = element_text(color = "black",size = 10),
        legend.position = "none")

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


### step2 数据合并

In [24]:
base_dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result/"
out_dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge/"

celltype_dirs <- list.dirs(base_dir, full.names = TRUE, recursive = FALSE)
celltype_dirs <- celltype_dirs[!grepl("\\.ipynb_checkpoints", celltype_dirs)]

In [25]:
celltype_dirs <- celltype_dirs[14]
celltype_dirs

[1] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Treg-FOXP3"

In [26]:
for(cell_dir in celltype_dirs){
  celltype <- basename(cell_dir)
  message("Processing celltype: ", celltype)
  
  gene_dirs <- list.dirs(cell_dir, full.names = TRUE, recursive = FALSE)
  gene_dirs <- gene_dirs[!grepl("\\.ipynb_checkpoints", gene_dirs)]  # 再次过滤
  
  all_data <- list()
  
  for(gene_dir in gene_dirs){
    gene <- basename(gene_dir)
    step2_file <- list.files(gene_dir, pattern = "_step2_output$", full.names = TRUE)
    
    if(length(step2_file) == 0){
      next  # 没有 _step2_output 文件，跳过
    }
    
    # 读取数据
    df <- tryCatch(
      read.delim(step2_file, header = TRUE, stringsAsFactors = FALSE, colClasses = "character"),
      error = function(e) NULL)
    
    # 如果为空表或读取失败，跳过
    if(is.null(df) || nrow(df) == 0){
      next
    }
    
    df$gene <- gene 
    df$celltype <- celltype
    all_data[[length(all_data) + 1]] <- df
        }
  
  # 合并该细胞类型的数据
  if(length(all_data) > 0){
    final_df <- bind_rows(all_data)
    
    # 保存为 csv
    out_file <- file.path(out_dir, paste0(celltype, "_step2_output.csv"))
    fwrite(final_df, out_file, row.names = FALSE)
    message("保存完成: ", out_file)
  } else {
    message("⚠️ 细胞类型 ", celltype, " 下没有找到任何有效的 _step2_output 文件")
  }
}

Processing celltype: CD4_Treg-FOXP3

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Treg-FOXP3_step2_output.csv



### step3 数据合并

In [8]:
base_dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result/"
out_dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge/"

celltype_dirs <- list.dirs(base_dir, full.names = TRUE, recursive = FALSE)
celltype_dirs <- celltype_dirs[!grepl("\\.ipynb_checkpoints", celltype_dirs)]

In [9]:
celltype_dirs

[1] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Naive_T-CCR7"  
 [2] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Naive_T-SOX4"  
 [3] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tcm-CXCR5"     
 [4] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tcm-GPR183"    
 [5] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tcm-IFIT3"     
 [6] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tcm-SOX4"      
 [7] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tem-CCR7neg"   
 [8] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Tfh_like-CXCR5"
 [9] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Th-TNFRSF11A"  
[10] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Th1-GZMK"      
[11] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Th17-RORC"     
[12] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Th22-CCR10"    
[13] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Treg-FCRL3"    
[14] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result//CD4_Treg-FOXP3"

In [10]:
for(cell_dir in celltype_dirs){
  celltype <- basename(cell_dir)
  message("Processing celltype: ", celltype)
  
  gene_dirs <- list.dirs(cell_dir, full.names = TRUE, recursive = FALSE)
  gene_dirs <- gene_dirs[!grepl("\\.ipynb_checkpoints", gene_dirs)]  # 再次过滤
  
  all_data <- list()
  
  for(gene_dir in gene_dirs){
    gene <- basename(gene_dir)
    step3_file <- list.files(gene_dir, pattern = "_genePval_output.csv$", full.names = TRUE)
    
    if(length(step3_file) == 0){
      next  # 没有 _step2_output 文件，跳过
    }
    
    # 读取数据
    df <- tryCatch(
      read.delim(step3_file, header = TRUE, stringsAsFactors = FALSE, colClasses = "character"),
      error = function(e) NULL)
    
    # 如果为空表或读取失败，跳过
    if(is.null(df) || nrow(df) == 0){
      next
    }
    
    df$celltype <- celltype
    all_data[[length(all_data) + 1]] <- df
        }
  
  # 合并该细胞类型的数据
  if(length(all_data) > 0){
    final_df <- bind_rows(all_data)
    
    # 保存为 csv
    out_file <- file.path(out_dir, paste0(celltype, "_genePval_output.csv"))
    fwrite(final_df, out_file, row.names = FALSE)
    message("保存完成: ", out_file)
  } else {
    message("⚠️ 细胞类型 ", celltype, " 下没有找到任何有效的 _genePval_output.csv 文件")
  }
}

Processing celltype: CD4_Naive_T-CCR7

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Naive_T-CCR7_genePval_output.csv

Processing celltype: CD4_Naive_T-SOX4

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Naive_T-SOX4_genePval_output.csv

Processing celltype: CD4_Tcm-CXCR5

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Tcm-CXCR5_genePval_output.csv

Processing celltype: CD4_Tcm-GPR183

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Tcm-GPR183_genePval_output.csv

Processing celltype: CD4_Tcm-IFIT3

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Tcm-IFIT3_genePval_output.csv

Processing celltype: CD4_Tcm-SOX4

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Tcm-SOX4_genePval_output.csv

Processing celltype: CD4_Tem-CCR7neg

保存完成: /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/result_merge//CD4_Tem-CCR7neg_gene